In [1]:
import pandas as pd
import openpyxl
from pandas import ExcelWriter

In [1]:
%store -r tarjetas
%store -r bancos

In [2]:
tarjeta_mastercard = tarjetas.loc [tarjetas['Cod Marca Producto']== 'Mastercard'] #Filtro por la Tarjeta Mastercard
tarjeta_mastercard = tarjeta_mastercard.loc[:, (tarjeta_mastercard!= 0).any(axis=0)]
tarjeta_mastercard = tarjeta_mastercard.loc[tarjetas['Cod Comercio']== 4547438] #Hay que filtrar por la tarjeta Mastercad y por este numero de comercio

In [3]:
tarjeta_mastercard["PERCEPCION IVA R.G. 2408  1,50/3,00 %"] = tarjeta_mastercard[
                  ['PERCEPCION IVA R.G. 2408   1,50 %', 'PERCEPCION IVA R.G. 2408   3,00 %']].sum(axis=1, min_count=1)
tarjeta_mastercard['VENTAS'] = tarjeta_mastercard['VENTAS C/DESCUENTO CONTADO'] + tarjeta_mastercard['VENTAS C/DTO CUOTAS FINANC. OTORG.']
tarjeta_mastercard['RETENCION ING.BRUTOS SIRTAC'] = (tarjeta_mastercard['RETENCION ING.BRUTOS SIRTAC']
                                                  - tarjeta_mastercard['AJUSTE SIRTAC NOV. 1RA QU']
                                                  + tarjeta_mastercard['QR  RETENCION IIBB BS. AS.']
                                                  + tarjeta_mastercard['AJUSTE SIRTAC OCT. 2DA QUIN'])  
tarjeta_mastercard['RETENCION IVA'] = tarjeta_mastercard['RETENCION IVA'] + tarjeta_mastercard['QR  RETENCION IVA']
tarjeta_mastercard['RETENCION IMP.GANANCIAS'] = tarjeta_mastercard['RETENCION IMP.GANANCIAS'] + tarjeta_mastercard['QR  RETENCION GANANCIAS']

tarjeta_mastercard["TOTAL DEDUCCIONES"] = tarjeta_mastercard[["TOTAL DEDUCCIONES", "TOTAL PAGOS DE COMERCIOS",'CARGOS DE ENTIDAD']
                                                             ].sum(axis=1, min_count=1)
tarjeta_mastercard["TOTAL DEDUCCIONES"]= tarjeta_mastercard["TOTAL DEDUCCIONES"] - tarjeta_mastercard['REEMBOLSOS DE ENTIDAD']
tarjeta_mastercard['IVA 21%'] = tarjeta_mastercard[['IVA CRED.FISC.COMERCIO S/ARANC 21,00%','IVA PROMO CUOTAS AHORA 12/18 - 21,00%'
                              , 'IVA RI SIST CUOTAS','IVA RI CRED.FISC.COMERCIO S/DTO F.OTORG'
                              ,'IVA RI. CARGO LIQUID. ELECT.', 'IVA RI SERV.OPER. INT.']].sum(axis=1, min_count=1)
tarjeta_mastercard['ARANCEL'] = (tarjeta_mastercard['ARANCEL'] + tarjeta_mastercard['CARGO  SISTEMA CUOTAS MENS'] +
                                tarjeta_mastercard['CARGO LIQUIDACION ELECTR.']+ tarjeta_mastercard['SERVICIO OPER. INTERNAC.'])
tarjeta_mastercard['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'] = (tarjeta_mastercard['DESCUENTO FINANC.OTORG. CUOTAS']
                                                                              + tarjeta_mastercard['PROMO CUOTAS AHORA 12/18'])
tarjeta_mastercard['Iva 10,5%'] = (tarjeta_mastercard['IVA PROMO CUOTAS AHORA 12/18 - 10,50%'] 
                               + tarjeta_mastercard['IVA CRED.FISC.COM.L.25063 S/DTO F.OTOR 10,50'])
tarjeta_mastercard = tarjeta_mastercard.reindex(columns=['Fecha de Pago', 'Cod Marca Producto', 'Cod Comercio',
       'Nro Liquidacion','VENTAS', 'ARANCEL', 'IVA 21%', 'DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18',
       'Iva 10,5%', 'RETENCION IVA','PER B.A.I.BR.DN.01/04', 'RETENCION IMP.GANANCIAS','RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC',"PERCEPCION IVA R.G. 2408  1,50/3,00 %",
       'TOTAL DEDUCCIONES', 'IMPORTE NETO DE PAGOS']) #reordeno columnas, aca ya quedan en el orden que es necesario,  ,
tarjeta_mastercard['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'] = tarjeta_mastercard['Iva 10,5%']*100/10.5
tarjeta_mastercard['ARANCEL'] = tarjeta_mastercard['IVA 21%']*100/21            

tarjeta_mastercard.insert(7, 'Iva 21 control', tarjeta_mastercard['IVA 21%']/tarjeta_mastercard['ARANCEL'])
                                #agrego columna de control para IVA 21%
tarjeta_mastercard.insert(10, 'Iva 10,5 control', 
                                tarjeta_mastercard['Iva 10,5%']/tarjeta_mastercard['DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18'])
                                #agrego columna de control para IVA 10,5%                                
tarjeta_mastercard = tarjeta_mastercard.assign(Diferencia = tarjeta_mastercard.iloc[:, 5:7].sum(axis=True) 
                         + tarjeta_mastercard.iloc[:, 8:10].sum(axis=True)
                         + tarjeta_mastercard.iloc[:, 11:18].sum(axis=True)
                         - tarjeta_mastercard['TOTAL DEDUCCIONES']) #agrego columna de control final
tarjeta_mastercard = tarjeta_mastercard.round(2)

In [4]:
tarjeta_mastercard.columns

Index(['Fecha de Pago', 'Cod Marca Producto', 'Cod Comercio',
       'Nro Liquidacion', 'VENTAS', 'ARANCEL', 'IVA 21%', 'Iva 21 control',
       'DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18', 'Iva 10,5%',
       'Iva 10,5 control', 'RETENCION IVA', 'PER B.A.I.BR.DN.01/04',
       'RETENCION IMP.GANANCIAS', 'RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC', 'PERCEPCION IVA R.G. 2408  1,50/3,00 %',
       'TOTAL DEDUCCIONES', 'IMPORTE NETO DE PAGOS', 'Diferencia'],
      dtype='object', name='Desc Concepto Clearing')

In [5]:
sum_columns = [
       'Nro Liquidacion', 'VENTAS', 'ARANCEL', 'IVA 21%', 'Iva 21 control',
       'DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18', 'Iva 10,5%',
       'Iva 10,5 control', 'RETENCION IVA', 'PER B.A.I.BR.DN.01/04',
       'RETENCION IMP.GANANCIAS', 'RETENCION ING.BRUTOS',
       'RETENCION ING.BRUTOS SIRTAC', "PERCEPCION IVA R.G. 2408  1,50/3,00 %",
       'TOTAL DEDUCCIONES',
       'IMPORTE NETO DE PAGOS', 'Diferencia'] #'REEMBOLSOS DE ENTIDAD',
max_columns = ['Nro Liquidacion']

group = {col: ('max' if col in max_columns else 'sum') for col in sum_columns}



In [6]:
tarjeta_mastercard=tarjeta_mastercard.groupby(by=['Cod Marca Producto', 'Cod Comercio']).agg(group)

In [8]:
tarjeta_mastercard= tarjeta_mastercard.drop(['Diferencia', 'Iva 21 control', 'Iva 10,5 control'], axis=1)

In [9]:
writer = ExcelWriter(r'C:\Users\Juan M Cabral\Desktop\Liquidaciones tarjetas\11-22\Mastercard1122.xlsx')
tarjeta_mastercard.to_excel(writer, 'Hoja de datos', index=True)
writer.save()

C:\Users\Juan M Cabral\AppData\Local\Temp\ipykernel_2500\191230414.py:3: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [ ]:


tarjeta_mastercard[(tarjeta_mastercard['REEMBOLSOS DE ENTIDAD'] > 1)]

In [7]:
tarjeta_mastercard

,Desc Concepto Clearing,Nro Liquidacion,VENTAS,ARANCEL,IVA 21%,Iva 21 control,DESCUENTO FINANC.OTORG. CUOTAS/PROMO CUOTAS AHORA 12/18,"Iva 10,5%","Iva 10,5 control",RETENCION IVA,PER B.A.I.BR.DN.01/04,RETENCION IMP.GANANCIAS,RETENCION ING.BRUTOS,RETENCION ING.BRUTOS SIRTAC,"PERCEPCION IVA R.G. 2408 1,50/3,00 %",TOTAL DEDUCCIONES,IMPORTE NETO DE PAGOS,Diferencia
Cod Marca Producto,Cod Comercio,,,,,,,,,,,,,,,,,
Mastercard,4547438,149215,1.727654e+08,3518321.09,738847.43,13.23,10010475.91,1051099.97,4.33,4777127.99,94700.96,1588633.84,3057420.15,1383225.93,108304.94,26328159.44,1.464373e+08,-1.25
